# 文本预处理


文本是一类序列数据，一篇文章可以看作是字符或单词的序列，本节将介绍文本数据的常见预处理步骤，预处理通常包括四个步骤：

1. 读入文本
2. 分词
3. 建立字典，将每个词映射到一个唯一的索引（index）
4. 将文本从词的序列转换为索引的序列，方便输入模型

## 读入文本

我们用一部英文小说，即H. G. Well的[Time Machine](http://www.gutenberg.org/ebooks/35)，作为示例，展示文本预处理的具体过程。

In [22]:
import collections
import re

def read_time_machine():
    with open('./data/input/timemachine.txt', 'r') as f:
        # 把所有非a-z的连续字符用空格替换
        lines = [re.sub('[^a-z]+', ' ', line.strip().lower()) for line in f]
    return lines


lines = read_time_machine()
print('# sentences %d' % len(lines))

# sentences 3221


In [23]:
lines[2000]

'once have been the favoured aristocracy and the morlocks their'

## 分词

我们对每个句子进行分词，也就是将一个句子划分成若干个词（token），转换为一个词的序列。

In [24]:
def tokenize(sentences, token='word'):
    """Split sentences into word or char tokens"""
    if token == 'word':
        return [sentence.split(' ') for sentence in sentences]
    elif token == 'char':
        return [list(sentence) for sentence in sentences]
    else:
        print('ERROR: unkown token type '+token)

tokens = tokenize(lines)
tokens[0:2]

[['the', 'time', 'machine', 'by', 'h', 'g', 'wells', ''], ['']]

## 建立字典

为了方便模型处理，我们需要将字符串转换为数字。因此我们需要先构建一个字典（vocabulary），将每个词映射到一个唯一的索引编号。

In [25]:
def count_corpus(sentences):  # 参数sentence 为二维数组
    tokens = [tk for st in sentences for tk in st]  # 先转成一个list, 再统计词频
    return collections.Counter(tokens)  # 返回一个字典，记录每个词的出现次数

In [26]:
class Vocab(object):
    def __init__(self, tokens, min_freq=0, use_special_tokens=False):
        counter = count_corpus(tokens)  # : 这里的tokens为词组成的二维数组。词组成的句子是list
        self.token_freqs = list(counter.items())
        self.idx_to_token = []
        if use_special_tokens:
            # padding, begin of sentence, end of sentence, unknown
            self.pad, self.bos, self.eos, self.unk = (0, 1, 2, 3)
            self.idx_to_token += ['', '', '', '']
        else:
            self.unk = 0
            self.idx_to_token += ['']
        # token_freqs is list with tuple that each contains 2 element. such as [(‘nice’, 666), ...]
        # 注意：这里的 token_freqs 不包含重复的元素
        self.idx_to_token += [token for token, freq in self.token_freqs
                        if freq >= min_freq and token not in self.idx_to_token]
        self.token_to_idx = dict()
        for idx, token in enumerate(self.idx_to_token):
            self.token_to_idx[token] = idx

    def __len__(self):
        return len(self.idx_to_token)

    def __getitem__(self, tokens):
        """
        token_to_index
        把词转成索引号
        
        isinstance() 函数来判断一个对象是否是一个已知的类型，类似 type()。
        
        isinstance() 与 type() 区别：
        
            type() 不会认为子类是一种父类类型，不考虑继承关系。
            isinstance() 会认为子类是一种父类类型，考虑继承关系。
        
        如果要判断两个类型是否相同推荐使用 isinstance()。
        """
        if not isinstance(tokens, (list, tuple)):  # 只要是后面(list, tuple) 中的一个就返回True
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    def to_tokens(self, indices):
        """
        index_to_token
        把索引号转成词
        """
        if not isinstance(indices, (list, tuple)):  # 只要是后面(list, tuple) 中的一个就返回True
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]


<font color=red size=5>
Python.__getitem__方法 -- 举个栗子🌰<br>
</font>
<br>

<font color=red size=3>
Python的魔法方法__getitem__ 可以让对象实现迭代功能，这样就可以使用for...in... 来迭代该对象了
</font>
<br>


In [36]:
class Animal:
    def __init__(self, animal_list):
        self.animals_name = animal_list

animals = Animal(["dog","cat","fish"])
for animal in animals:
    print(animal)

TypeError: 'Animal' object is not iterable

<font color=red size=3>
在用 for..in.. 迭代对象时，如果对象没有实现 __iter__ __next__ 迭代器协议，Python的解释器就会去寻找__getitem__ 来迭代对象，如果连__getitem__ 都没有定义，这解释器就会报对象不是迭代器的错误： 如上所示
</font>
<br>

```python
TypeError: 'Animal' object is not iterable
```


<font color=red size=3>
而实现这个方法后，就可以正常迭代对象了
</font>
<br>

In [50]:
class Animal:
    def __init__(self, animal_list):
        self.animals_name = animal_list

    def __getitem__(self, index):
        return self.animals_name[index]

animals = Animal(["dog","cat","fish"])
for animal in animals:
    print(animal)

dog
cat
fish


In [51]:
animals.animals_name

['dog', 'cat', 'fish']

In [52]:
type(animals.animals_name)

list

In [53]:
animals[1]

'cat'

In [54]:
arr = ['a', 'b', 'c']
arr.index('a')

0

## 将词转为索引

使用字典，我们可以将原文本中的句子从单词序列转换为索引序列

In [55]:
for i in range(8, 10):
    print('words:', tokens[i])  # token 为二维数组[['nice', 'to', 'meet', 'you'], ...]
    print('indices:', vocab[tokens[i]])

words: ['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him', '']
indices: [1, 2, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 0]
words: ['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
indices: [20, 21, 22, 23, 24, 16, 25, 26, 27, 28, 29, 30]


我们看一个例子，这里我们尝试用Time Machine作为语料构建字典

In [56]:
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[0:10])

[('', 0), ('the', 1), ('time', 2), ('machine', 3), ('by', 4), ('h', 5), ('g', 6), ('wells', 7), ('i', 8), ('traveller', 9)]


In [57]:
type(vocab.idx_to_token)

list

In [58]:
vocab.idx_to_token[:10]

['', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells', 'i', 'traveller']

In [59]:
type(vocab.token_to_idx)

dict

# 测试 `__getitem__()` 和 `to_tokens()` 方法

In [60]:
# self.unk 的index为0, 未登录词都被标记为 self.unk
vocab.__getitem__(['mengfuli', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells', 'i', 'traveller'])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [61]:
# 
vocab.to_tokens([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

['', 'the', 'time', 'machine', 'by', 'h', 'g', 'wells', 'i', 'traveller']

In [62]:
vocab[['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient']]  # word to index

[1, 2, 9, 10, 11, 12, 13, 14, 15]

## 用现有工具进行分词

我们前面介绍的分词方式非常简单，它至少有以下几个缺点:

1. 标点符号通常可以提供语义信息，但是我们的方法直接将其丢弃了
2. 类似“shouldn't", "doesn't"这样的词会被错误地处理
3. 类似"Mr.", "Dr."这样的词会被错误地处理

我们可以通过引入更复杂的规则来解决这些问题，但是事实上，有一些现有的工具可以很好地进行分词，我们在这里简单介绍其中的两个：[spaCy](https://spacy.io/)和[NLTK](https://www.nltk.org/)。

下面是一个简单的例子：

In [63]:
text = "Mr. Chen doesn't agree with my suggestion."

spaCy:

In [78]:
# import spacy
# nlp = spacy.load('en_core_web_sm')
# doc = nlp(text)
# print([token.text for token in doc])

# 输出 ['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']

In [77]:
# python -m spacy download en_core_web_sm

In [76]:
spacy.__version__

'2.2.3'

In [75]:
# !pip install -i https://pypi.tuna.tsinghua.edu.cn/simple spacy

NLTK:

In [79]:
# from nltk.tokenize import word_tokenize
# from nltk import data
# data.path.append('./data/input/nltk_data')  # 数据600+M，这里没下载
# print(word_tokenize(text))

# ['Mr.', 'Chen', 'does', "n't", 'agree', 'with', 'my', 'suggestion', '.']